In [ ]:
#%%
print('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting')

from array import array
from cmath import nan
from pyexpat import model
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms
from torch.autograd import variable
from itertools import chain
from sklearn import metrics as met
import pickle
from icecream import ic

import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
# import util
# import model_torch_simple
# from torchmetrics import Accuracy
from tqdm import tqdm
import argparse
from icecream import ic
import numpy as np
from PIL import Image
device = 'cuda' if torch.cuda.is_available() else 'cpu'
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score
#%%
seed = 42
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)

# train_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_train_gene.csv', delimiter = ',')
# train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_train_hml.csv')
# train_target = train_target[['EMB_MIC']]
# # don't touch test data, split out validation data from training data during training
# # test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_EMB/aa_data_test_pca4k.csv', delimiter = ',')
# test_data = np.loadtxt('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/aa_data_test_gene.csv', delimiter = ',')
# test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_new_emb/mic_aa_test_hml.csv')
# test_target = test_target[['EMB_MIC']]

# all_data = np.concatenate((train_data, test_data), axis=0)
# all_target = pd.concat((train_target, test_target), axis=0)

# train_data, test_data, train_target, test_target = train_test_split(all_data, all_target, test_size=0.2, random_state=42, stratify=all_target)

/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/nn_model_script_emb_test.py - starting


In [43]:
def data_split(aa_array, encoded_mic):
    # Encode the target variable
    # Perform stratified train-test split
    train_data, test_data, train_target, test_target = train_test_split(
        aa_array,
        encoded_mic,
        test_size=0.1,  # 10% for testing
        stratify=encoded_mic,  # Ensures the proportion of each class is preserved
        random_state=42  # For reproducibility
    )
    return train_data, test_data, train_target, test_target
def is_within_doubling_dilution(pred, target, target_min, target_max):
    _ = np.arange(target_min-1, target_max+2, 1)
    index = [i for i, x in enumerate(_) if x == target][0]
    return (_[index-1] <= pred <= _[index+1])
def data_prep_cryptic(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
        else:
            aa.append([0]*len(all_snp))
        
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1
    
    mic_aa = cryptic[cryptic['ENA_RUN'].isin(overlap)]#.iloc[:,14:27]
    # mic_aa = cryptic[cryptic['ENA_RUN'].isin(variants['sample_id'].unique())]#.iloc[:,14:27]
    # print(mic_aa.shape)
    # mic_aa['wgs_id'] = pd.Categorical(mic_aa['ENA_RUN'], categories=variants['sample_id'].unique().tolist(), ordered=True)
    # mic_aa = mic_aa.sort_values('ENA_RUN')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
    mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)
    # mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(variants['sample_id'].unique().tolist())
    mic_aa = mic_aa.sort_values(["ENA_RUN"])  ## 'sort' changed to 'sort_values'
    
    return aa_array, mic_aa

def data_prep_50k(cryptic, gene_list, dr_list):
    # overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    # variants = variants[variants['sample_id'].isin(overlap)]
    # variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = pd.read_csv('../variants_full.csv')
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']

    overlap = set(variants['sample_id']).intersection(set(cryptic['ENA_RUN'].to_list()))
    # variants = variants[variants['drugs'].isin(['ethambutol'])]
    variants = variants[variants['gene'].isin(gene_list)]
    variants = variants[variants['sample_id'].isin(overlap)]
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    # crypticSNPnames = np.load('crypticSNPnames.npy', allow_pickle=True)
    # variants = variants[variants['SNP'].isin(crypticSNPnames)]
    print(variants['SNP'].unique().shape)
    # print(variants.shape)
    # print(variants['sample_id'].unique().shape)
    def compare_snp_lists_with_values_optimized(set_list, query_list, values_list):
        # Create a dictionary from query_list and values_list for direct mapping
        query_dict = dict(zip(query_list, values_list))
        # Use list comprehension to build the output list directly
        output_list = [query_dict.get(snp, 0) for snp in set_list]
        return output_list
    # Example usage
    # set_list = ['SNP1', 'SNP2', 'SNP3', 'SNP4']
    # query_list = ['SNP2', 'SNP4']
    # values_list = [5, 10]  # Corresponding values for 'SNP2' and 'SNP4'
    # output_list = compare_snp_lists_with_values_optimized(set_list, query_list, values_list)
    # print(output_list)  # Expected output: [0, 5, 0, 10]# Getting all snp data
    aa = []
    dr = []
    
    all_snp = variants['SNP'].unique() # here is a list of all snps values title for the row in the final table
    np.save('all_snp_emb.npy', all_snp)
    print(len(all_snp))
### 50k data reload
    variants = pd.read_csv('../variants_full.csv')
    variants = variants[variants['gene'] != 'PPE35']
    variants = variants[variants['type'] != 'synonymous_variant']
    variants = variants[variants['type'] != 'non_coding_transcript_exon_variant']
    variants['SNP'] = variants['gene'] + '-'+ variants['change']
    variants = variants[variants['SNP'].isin(all_snp)]
    # certify we want 50k data that doesn't overlap with cryptic
    overlap = variants[~variants['sample_id'].isin(cryptic['ENA_RUN'])]
    overlap = overlap['sample_id'].unique()

    for x in tqdm(overlap):
    # for x in tqdm(variants['sample_id'].unique()):
        if x in variants['sample_id'].tolist():
            aa.append(compare_snp_lists_with_values_optimized(all_snp, variants[variants['sample_id']==x]['SNP'].to_list(), variants[variants['sample_id']==x]['freq'].to_list()))
            if dr_list[0] in variants[variants['sample_id']==x]['drugs'].unique():
                dr.append(1)
            else:
                dr.append(0)
        else:
            # aa.append([0]*len(all_snp))
            pass
        # print('SNP')
    aa_array = np.array(aa)
    aa_array[aa_array < 0.8] = 0
    aa_array[aa_array >= 0.8] = 1

    return aa_array, dr

### RIF

In [ ]:
drug = 'RIF'
dr_list = ['rifampicin']

df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
gene_list = ['rpoA', 'rpoB', 'rpoC']
# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '<=0.06'])]
df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '0.06','<=0.06'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0','4.0', '2.0', '1.0', '0.5', '0.25', '0.12', '0.06','<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>4' :
        df_emb.loc[i, f'{x}'] = '8'
    elif row[x] == '>8':
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
        # df_emb.loc[i, f'{x}'] = '0.03'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.03125'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

/tmp/ipykernel_27943/2489226644.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb.loc[i, f'{x}'] = '0.03125'
/tmp/ipykernel_27943/2489226644.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb.loc[i, f'{x}'] = '0.125'
/tmp/ipykernel_27943/2489226644.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb.loc[i, f'{x}'] = '8'
/tmp/ipykernel_27943/2489226644.py:17: SettingWithCopyWarning: 
A value is trying to be se

(1253,)
1253


100%|██████████| 9176/9176 [00:46<00:00, 198.26it/s]


(1253,)
1253


100%|██████████| 103974/103974 [2:18:36<00:00, 12.50it/s] 


### ETH


In [55]:
drug = 'ETH'
dr_list = ['ethambutol']
gene_list = ['ethA', 'inhA']
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
# df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '0.25', '<=0.25'])]
df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.25'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

/tmp/ipykernel_27943/2868685399.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb.loc[i, f'{x}'] = '16'
/tmp/ipykernel_27943/2868685399.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')


(997,)
997


100%|██████████| 11539/11539 [00:06<00:00, 1679.22it/s]
/tmp/ipykernel_27943/2469696435.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_27943/2469696435.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(997,)
997


100%|██████████| 33769/33769 [10:03<00:00, 55.97it/s]


### INH


In [ ]:
drug = 'INH'
gene_list = ['inhA', 'katG','kasA']
dr_list = ['isnoiazid']
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
# df_emb = df[df[f'{drug}_MIC'].isin(['>1.6','0.8', '0.4', '0.2', '0.1', '0.05', '<=0.025'])]
df_emb = df[df[f'{drug}_MIC'].isin(['>1.6','1.6','0.8', '0.4', '0.2', '0.1', '0.05', '<=0.025'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1.6' :
        df_emb.loc[i, f'{x}'] = '3.2'
    elif row[x] == '<=0.025':
        # df_emb.loc[i, f'{x}'] = '0.0125'
        df_emb.loc[i, f'{x}'] = '0.025'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

/tmp/ipykernel_27943/560431133.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb.loc[i, f'{x}'] = '0.025'
/tmp/ipykernel_27943/560431133.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb.loc[i, f'{x}'] = '3.2'
/tmp/ipykernel_27943/560431133.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')


(932,)
932


100%|██████████| 10088/10088 [00:24<00:00, 408.70it/s]
/tmp/ipykernel_27943/2469696435.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_27943/2469696435.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(932,)
932


 43%|████▎     | 40815/95514 [33:39<46:22, 19.66it/s]  

### RFB


In [ ]:
drug = 'RFB'
res_thresh = 0.12
dr_list = ['rifabutin']
gene_list = ['rpoA', 'rpoB', 'rpoC']
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12', '<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>2' :
        df_emb.loc[i, f'{x}'] = '4'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.03125'
        
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
    
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

/tmp/ipykernel_27943/3524328196.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb.loc[i, f'{x}'] = '4'
/tmp/ipykernel_27943/3524328196.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb.loc[i, f'{x}'] = '0.125'
/tmp/ipykernel_27943/3524328196.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')


(1023,)
1023


100%|██████████| 4694/4694 [00:15<00:00, 306.58it/s]
/tmp/ipykernel_27943/2469696435.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.astype('category')
/tmp/ipykernel_27943/2469696435.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mic_aa.ENA_RUN = mic_aa.ENA_RUN.cat.set_categories(overlap)


(1023,)
1023


 20%|██        | 21327/106296 [27:17<1:48:45, 13.02it/s]


KeyboardInterrupt: 

### MXF


In [ ]:
drug = 'MXF'
dr_list = ['moxifloxacin']
gene_list = ['gyrA', 'gyrB']
res_thresh = 1
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df_emb = df[df[f'{drug}_MIC'].isin(['>4','4.0', '2.0', '1.0', '0.5', '0.25', '0.12'])]
# df_emb = df[df[f'{drug}_MIC'].isin(['>4','4', '2', '1', '0.5', '0.25', '0.12', '<=0.06'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>4' :
        df_emb.loc[i, f'{x}'] = '8'
    elif row[x] == '<=0.06':
        df_emb.loc[i, f'{x}'] = '0.03'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float') 
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

### KAN 


In [ ]:
drug = 'KAN'
dr_list = ['kanamycin']
gene_list = ['rrs', 'eis']
res_thresh = 4
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df_emb = df[df[f'{drug}_MIC'].isin(['>16','16.0', '8.0', '4.0', '2.0', '<=1'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>16' :
        df_emb.loc[i, f'{x}'] = '32'
    elif row[x] == '<=1':
        df_emb.loc[i, f'{x}'] = '1.0'
        # df_emb.loc[i, f'{x}'] = '0.5'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

### LEV


In [ ]:
drug = 'LEV'
# gene_list = ['pgi', 'fadE4', 'php', 'cyp132', 'pckA', 'rpmB1', 'pfkB', 'acg', 'ctpF', 'cyp132', 'pckA', 'pfk']
# gene_list = np.unique(variants['gene'])
dr_list = ['levofloxacin']
gene_list = ['gyrA', 'gyrB']
res_thresh = 1
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '0.25', '<=0.12'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
        # df_emb.loc[i, f'{x}'] = '0.06'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

### AMI


In [ ]:
# gene_list = ['pgi', 'fadE4', 'php', 'cyp132', 'pckA', 'rpmB1', 'pfkB', 'acg', 'ctpF', 'cyp132', 'pckA', 'pfk']
# gene_list = np.unique(variants['gene'])
dr_list = ['amycacin']
drug = 'AMI'
# gene_list = np.unique(variants['gene'])
gene_list = ['rrs', 'eis']

res_thresh = 1
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df_emb = df[df[f'{drug}_MIC'].isin(['>8','8.0', '4.0', '2.0', '1.0', '0.5', '<=0.25'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>8' :
        df_emb.loc[i, f'{x}'] = '16'
    elif row[x] == '<=0.25':
        df_emb.loc[i, f'{x}'] = '0.25'
        # df_emb.loc[i, f'{x}'] = '0.125'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

### BDQ



In [ ]:
drug = 'BDQ'
gene_list = [ 'Rv0678', 'atpE', 'pepQ']
res_thresh = 0.25

df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df_emb = df[df[f'{drug}_MIC'].isin(['>1', '1.0', '0.5', '0.25', '0.12', '0.06', '0.03', '<=0.015'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1' :
        df_emb.loc[i, f'{x}'] = '2'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.03125'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

### CFZ


In [ ]:
drug = 'CFZ'
dr_list = ['clofazimine']
gene_list = np.unique(variants['gene'])
gene_list = [ 'Rv0678', 'pepQ']
res_thresh = 0.25
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df_emb = df[df[f'{drug}_MIC'].isin(['>2','2.0', '1.0', '0.5', '0.25', '0.12', '0.06', '0.03', '<=0.015'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>1' :
        df_emb.loc[i, f'{x}'] = '2'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.03125'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))

### DLM


In [ ]:
drug = 'DLM'
dr_list = ['delamanid']
res_thresh = 0.12
gene_list = ['fgd1','fbiA','fbiB','fbiC','fbiD','ddn']
gene_list = variants['gene'].unique()
df = pd.read_csv('../CRyPTIC_reuse_table_20231208.csv')
df_emb = df[df[f'{drug}_MIC'].isin(['>0.5', '0.5', '0.25', '0.12', '0.06', '0.03', '0.015', '<=0.015'])]
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]
for i, row in df_emb.iterrows():
    x = f'{drug}_MIC'
    if row[x] == '>0.5' :
        df_emb.loc[i, f'{x}'] = '1'
    elif row[x] == '<=0.015':
        df_emb.loc[i, f'{x}'] = '0.0078125'
    elif row[x] == '0.12':
        df_emb.loc[i, f'{x}'] = '0.125'        
    elif row[x] == '0.06':
        df_emb.loc[i, f'{x}'] = '0.0625'
    elif row[x] == '0.03':
        df_emb.loc[i, f'{x}'] = '0.03125'
    elif row[x] == '0.015':
        df_emb.loc[i, f'{x}'] = '0.015625'        
        
df_emb[f'{drug}_MIC'] = df_emb[f'{drug}_MIC'].astype('float')
# df_emb = df_emb[~df_emb['ENA_RUN'].isin(to_be_dropped)]

# variants = pd.read_csv('variants_full.csv')
# variants = variants[variants['type'] != 'synonymous_variant']
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['LOW','MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[~df_emb['EMB_PHENOTYPE_QUALITY'].isin(['MEDIUM'])]  # remove low and med quality
# df_emb = df_emb[df_emb['ENA_RUN'].isin(samples)]
cryptic = df_emb
aa_array_cryptic, drs_cryptic = data_prep_cryptic(cryptic, gene_list,dr_list)
aa_array_50k, drs_50k = data_prep_50k(cryptic, gene_list,dr_list)

# encoded_mic = np.array([0 if value < 4 else 1 for value in mic_aa['EMB_MIC'].to_list()])

with open(f'./generated_data18122024/all_sample_snps_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_cryptic)
with open(f'./generated_data18122024/all_sample_drs_cryptic_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_cryptic[f'{drug}_MIC']))
with open(f'./generated_data18122024/all_sample_snps_50k_{drug}.npy', 'wb') as f:
    np.save(f, aa_array_50k)
with open(f'./generated_data18122024/all_sample_drs_50k_{drug}.npy', 'wb') as f:
    np.save(f, np.array(drs_50k))